In [1]:
using SatelliteToolbox
using Dates

In [2]:
function to_datatime(jd)
    date_tuple = JDtoDate(jd)
    
    DateTime(
        date_tuple[1],
        date_tuple[2],
        date_tuple[3],
        date_tuple[4],
        date_tuple[5],
        round(date_tuple[6]),
    )
end

to_datatime (generic function with 1 method)

In [3]:
tle_str = """
CUBEL                   
1 47448U 21006AM  21104.70471889  .00000294  00000-0  20468-4 0  9997
2 47448  97.5032 167.1437 0011425 327.0810  32.9708 15.12771589 12411
"""
tle = read_tle_from_string(tle_str)[1]

days = 30;

In [4]:
orbp = init_orbit_propagator(Val{:J4}, tle)
T = 24*60*60 / tle.n

_, r, v = propagate!(orbp, 0:T*days);


t₁ = tle.epoch
t₂ = t₁ + T*days / (24*60*60)
r_sun = map(t -> sun_position_i(t), range(t₁, t₂, length=length(1:T*days)));

illumination = map(satellite_lighting_condition, r, r_sun);

In [5]:
illumnication_change = findall(v -> abs(v) > 0, diff(illumination))

_epoch = to_datatime(tle.epoch)
println("EPOCH: ", _epoch)
println("##########")
for c ∈ illumnication_change
    
    ts = _epoch + Dates.Second(c)
    if illumination[c-1] == SAT_LIGHTING_SUNLIGHT
       println("From sunlight to eclipse @ ", ts) 
    end
    if illumination[c-1] == SAT_LIGHTING_UMBRA
       println("From eclipse to sunlight @ ", ts) 
    end
end

EPOCH: 2021-04-14T16:54:48
##########
From eclipse to sunlight @ 2021-04-14T17:06:48
From sunlight to eclipse @ 2021-04-14T18:08:53
From eclipse to sunlight @ 2021-04-14T18:42:03
From sunlight to eclipse @ 2021-04-14T19:44:08
From eclipse to sunlight @ 2021-04-14T20:17:18
From sunlight to eclipse @ 2021-04-14T21:19:22
From eclipse to sunlight @ 2021-04-14T21:52:32
From sunlight to eclipse @ 2021-04-14T22:54:37
From eclipse to sunlight @ 2021-04-14T23:27:47
From sunlight to eclipse @ 2021-04-15T00:29:51
From eclipse to sunlight @ 2021-04-15T01:03:02
From sunlight to eclipse @ 2021-04-15T02:05:06
From eclipse to sunlight @ 2021-04-15T02:38:16
From sunlight to eclipse @ 2021-04-15T03:40:20
From eclipse to sunlight @ 2021-04-15T04:13:31
From sunlight to eclipse @ 2021-04-15T05:15:35
From eclipse to sunlight @ 2021-04-15T05:48:45
From sunlight to eclipse @ 2021-04-15T06:50:49
From eclipse to sunlight @ 2021-04-15T07:24:00
From sunlight to eclipse @ 2021-04-15T08:26:04
From eclipse to sunlig